##### This notebook shows how to generate the ERDS maps and plots based on motor imagery epochs pre-processed with ASR and produce a create a report

###### It requires the .fif epochs file output from the "epochs_ASR" notebook


In [1]:
import mne 
import numpy as np
import matplotlib.pyplot as plt

from utils import create_name
from utils import plot_TF_M
from utils import plot_TF_M_stat
from utils import plot_ERDs

mne.set_log_level(verbose='WARNING')

%matplotlib qt

c:\Users\andresfs\OneDrive - NTNU\Research\Github\ERDS_Maps\utils.py:109: SyntaxWarning: invalid escape sequence '\s'
  fname = folder + '\s' + '{:02d}'.format(subject) + '.mat'
c:\Users\andresfs\OneDrive - NTNU\Research\Github\ERDS_Maps\utils.py:111: SyntaxWarning: invalid escape sequence '\s'
  fname = folder + '\s' + '{:02d}'.format(subject) + '_' + type + '.fif'


In [2]:
# Read the ASR preprocessed epochs

# Define the subject and the folder where the data is located
subject = 41
data_folder = r"C:\Users\andresfs\OneDrive - NTNU\Datasets\Dataset MI - GigaScience\Derivates"

# Read the ASR processed epochs
fname_mat = create_name(subject,data_folder,type='ASR_epo')
epochs_ASR = mne.read_epochs(fname_mat,preload=True)

In [3]:
# Check the epochs info
epochs_ASR?

Type:        EpochsFIF
String form:
<EpochsFIF |  200 events (all good), -1.94922 – 4.94922 s, baseline off, ~345.1 MB, data loaded,
 'MI_Right': 100
 'MI_Left': 100>
Length:      200
File:        c:\users\andresfs\.conda\envs\mne\lib\site-packages\mne\epochs.py
Docstring:  
Epochs read from disk.

Parameters
----------

fname : path-like | file-like
    The epochs to load. If a filename, should end with ``-epo.fif`` or
    ``-epo.fif.gz``. If a file-like object, preloading must be used.

proj : bool | 'delayed'
    Apply SSP projection vectors. If proj is 'delayed' and reject is not
    None the single epochs will be projected before the rejection
    decision, but used in unprojected state if they are kept.
    This way deciding which projection vectors are good can be postponed
    to the evoked stage without resulting in lower epoch counts and
    without producing results different from early SSP application
    given comparable parameters. Note that in this case baselining,
    d

In [4]:
# Prepare Epochs for TF (Time-frequency) analysis
channels = ['C3','C4']
epochs_TF_ASR = epochs_ASR.copy().apply_proj().pick_channels(channels)

# Compute TF Maps - Wavelet
freqs = np.arange(2, 35.5, 0.5)
n_cycles = freqs / 2.0  # different number of cycle per frequency
TF_ASR_wav = mne.time_frequency.tfr_morlet(epochs_TF_ASR, freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
TF_ASR_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_ASR, freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)


C:\Users\andresfs\AppData\Local\Temp\ipykernel_12612\1352617266.py:8: RuntimeWarning: joblib not installed. Cannot run in parallel.
  TF_ASR_wav = mne.time_frequency.tfr_morlet(epochs_TF_ASR, freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
C:\Users\andresfs\AppData\Local\Temp\ipykernel_12612\1352617266.py:9: RuntimeWarning: joblib not installed. Cannot run in parallel.
  TF_ASR_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_ASR, freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)


In [5]:
# Plot TF Maps
event_ids = ['MI_Right','MI_Left'] # events 
kwargs = dict(baseline=(-1,0), mode='percent', vmin=-1.2, vmax=1.2, fmin=None, fmax=None, cmap='jet')
map_ASR_wav = plot_TF_M(TF_ASR_wav, event_ids, **kwargs, comment='- ERDS - ASR - Wavelet - s' + str(subject), reporting=True)
map_ASR_mtaper = plot_TF_M(TF_ASR_mtaper, event_ids, **kwargs, comment='- ERDS - ASR - Multitaper - s' + str(subject), reporting=True)


In [10]:
# Plot ERD per frequency bands
ERD_ASR_wav = plot_ERDs(TF_ASR_wav,channels, comment=' - ASR - Wavelet - s' + str(subject), reporting=True)
ERD_ASR_mtaper = plot_ERDs(TF_ASR_mtaper,channels, comment=' - ASR - Multitaper - s' + str(subject), reporting=True)


In [9]:
rname = "Report s" + str(subject) + " - ERDS Plots"
report = mne.Report(title=rname)

# Add to report
for fig in map_ASR_wav:
    report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))
for fig in map_ASR_mtaper:
    report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))


# Add to report
ERD_ASR_wav.savefig('temp.png')
report.add_image(image='temp.png', title='ERDS mu/alpha & beta', section='TF - ERDs Maps', tags='s'+str(subject))
ERD_ASR_mtaper.savefig('temp.png')
report.add_image(image='temp.png', title='ERDS mu/alpha & beta', section='TF - ERDs Maps', tags='s'+str(subject))


# Save Report
report.save(rname + ".hdf5", overwrite=True)
report.save(rname + ".html", overwrite=True)

'C:\\Users\\andresfs\\OneDrive - NTNU\\Research\\Github\\ERDS_Maps\\Report s41 - ERDS Plots.html'